<a href="https://colab.research.google.com/github/MashRiza/ReadingFromKaggle/blob/main/ReadingFromKaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import polars as pl

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

# Upload kaggle.json
# you have to gey this file from your kaggle account
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rezamm79","key":"7ef569400d1f02fd5042417915a247ab"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# an example of kaggle api for data
# you can find other data api in kaggle
!kaggle competitions download -c child-mind-institute-detect-sleep-states

child-mind-institute-detect-sleep-states.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Replace 'zip_file_name.zip' with the actual name of your zip file
!unzip child-mind-institute-detect-sleep-states.zip


Archive:  child-mind-institute-detect-sleep-states.zip
  inflating: sample_submission.csv   
  inflating: test_series.parquet     
  inflating: train_events.csv        
  inflating: train_series.parquet    


In [ ]:

# Specify the path to your Parquet file
parquet_file_path = '/content/train_series.parquet'

# Read the Parquet file into a pandas DataFrame
df = pl.read_parquet(parquet_file_path)

# Display the DataFrame
print(df.head())

shape: (5, 5)
┌──────────────┬──────┬──────────────────────────┬────────┬────────┐
│ series_id    ┆ step ┆ timestamp                ┆ anglez ┆ enmo   │
│ ---          ┆ ---  ┆ ---                      ┆ ---    ┆ ---    │
│ str          ┆ u32  ┆ str                      ┆ f32    ┆ f32    │
╞══════════════╪══════╪══════════════════════════╪════════╪════════╡
│ 038441c925bb ┆ 0    ┆ 2018-08-14T15:30:00-0400 ┆ 2.6367 ┆ 0.0217 │
│ 038441c925bb ┆ 1    ┆ 2018-08-14T15:30:05-0400 ┆ 2.6368 ┆ 0.0215 │
│ 038441c925bb ┆ 2    ┆ 2018-08-14T15:30:10-0400 ┆ 2.637  ┆ 0.0216 │
│ 038441c925bb ┆ 3    ┆ 2018-08-14T15:30:15-0400 ┆ 2.6368 ┆ 0.0213 │
│ 038441c925bb ┆ 4    ┆ 2018-08-14T15:30:20-0400 ┆ 2.6368 ┆ 0.0215 │
└──────────────┴──────┴──────────────────────────┴────────┴────────┘


In [ ]:
df.shape[0]

127946340

In [ ]:

# Define the size of each chunk
chunk_size = 20000000  # Adjust this based on your requirements ( it is number of rows )

# Calculate the number of chunks
num_chunks = (df.shape[0] + chunk_size - 1) // chunk_size

# Iterate through chunks and write to new Parquet files
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, df.shape[0])

    # Create a lazy computation plan for the chunk
    chunk_plan = df.slice(start_idx, end_idx).lazy()

    # Execute the plan and get the resulting DataFrame
    chunk_df = chunk_plan.collect()

    # Specify the path for the new Parquet file (adjust the naming convention as needed)
    new_parquet_file_path = f'chunk_{i + 1}.parquet'

    # Write the chunk DataFrame to a new Parquet file
    chunk_df.write_parquet(new_parquet_file_path)


In [ ]:
# Downloading segmented files if needed
from google.colab import files

# List of file names
file_names = ['/content/chunk_1.parquet', '/content/chunk_2.parquet', '/content/chunk_3.parquet', '/content/chunk_4.parquet', '/content/chunk_5.parquet', '/content/chunk_6.parquet', '/content/chunk_7.parquet']

# Loop through the file names and download each file
for colab_file_path in file_names:
    files.download(colab_file_path)

In [ ]:
# you can convert polars dataframe to pandas dataframe
# Convert Polars DataFrame to Pandas DataFrame
pandas_df = df.to_pandas()

In [ ]:
# or you can download segmented files and use them
df = pd.read_parquet(parquet_file_path)